# Similarity-based Playlist Generation

In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import IPython.display as ipd

In [2]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

loaders_by_format = {
    "csv": pd.read_csv,
    "xlsx": pd.read_excel,
    "db": pd.read_sql,
    "parquet": pd.read_parquet,
    "npy": lambda x: pd.DataFrame(list(np.load(x, allow_pickle=True))),
}


def load_data(filepath):
    extension = Path(filepath).name.split(".")[1]
    return loaders_by_format[extension](filepath)


df = load_data("../data/processed/20250212032022_audio_features.npy")
df["track"] = df["filepath"].apply(lambda x: Path(x).name)
df["genre"] = df["style_genre_discogs400"].apply(lambda x: x.split("---")[0])
df["style"] = df["style_genre_discogs400"].apply(lambda x: x.split("---")[-1])

In [3]:
df.columns

Index(['extracted_time_utc', 'filepath', 'duration', 'sample_rate',
       'num_channels', 'md5', 'bit_rate', 'codec', 'tempocnn_bpm',
       'key_temperley_predict', 'key_temperley_probability',
       'key_krumhansl_predict', 'key_krumhansl_probability',
       'key_edma_predict', 'key_edma_probability',
       'loudness_ebur128_integrated_lufs', 'loudness_ebur128_range_lu',
       'emomusic_valence', 'emomusic_arousal', 'musicnn_embeddings_mean',
       'style_genre_discogs400', 'style_genre_discogs400_probability',
       'vggish_voice', 'vggish_instrumental', 'effnet_discogs_danceable',
       'effnet_discogs_not_danceable', 'discogs_embeddings_mean', 'track',
       'genre', 'style'],
      dtype='object')

In [4]:
# get 10 random tracks
sample = df.sample(3)[["track", "key_temperley_predict", "genre", "style", "filepath", "sample_rate"]]
for track in sample.to_dict(orient="records"):
    fpath = track["filepath"]
    sr = track["sample_rate"]
    # ipd.display(ipd.Audio(fpath, rate=sr))
sample

,track,key_temperley_predict,genre,style,filepath,sample_rate
704,2nLtzopw4rPReszdYBJU6h.mp3,A major,Rock,Nu Metal,../data/raw/MusAV/audio_chunks/audio.002/2n/2n...,44100.0
370,4e0yKeT7bzGAGtojzb7kKK.mp3,B major,Rock,Hard Rock,../data/raw/MusAV/audio_chunks/audio.003/4E/4e...,44100.0
842,708jkkXTxgf6oLHOa7IbaZ.mp3,A major,Electronic,Ambient,../data/raw/MusAV/audio_chunks/audio.002/70/70...,44100.0


In [5]:
query_track = "3RLV9wC6HBmfB3Vicwejc2.mp3"
query_discogs_embeddings = df[df["track"] == query_track]["discogs_embeddings_mean"].values[0]
query_musicnn_embeddings = df[df["track"] == query_track]["musicnn_embeddings_mean"].values[0]

df_query = df[["filepath", "track", "discogs_embeddings_mean", "musicnn_embeddings_mean"]].copy()

df_query["cosine_discogs_similarity"] = df_query["discogs_embeddings_mean"].apply(lambda x: cosine_similarity(x, query_discogs_embeddings))
df_query["cosine_musicnn_similarity"] = df_query["musicnn_embeddings_mean"].apply(lambda x: cosine_similarity(x, query_musicnn_embeddings))

In [6]:
# top 5 similar tracks based on discogs embeddings
discogs_results = df_query.sort_values("cosine_discogs_similarity", ascending=False).head(5)[["filepath", "track", "cosine_discogs_similarity"]]
musicnn_results = df_query.sort_values("cosine_musicnn_similarity", ascending=False).head(5)[["filepath", "track", "cosine_musicnn_similarity"]]

for result in [discogs_results, musicnn_results]:
    for track in result.to_dict(orient="records"):
        fpath = track["filepath"]
        # ipd.display(ipd.Audio(fpath, rate=sr))
    print(result)

                                               filepath  \
1096  ../data/raw/MusAV/audio_chunks/audio.005/3r/3R...   
1635  ../data/raw/MusAV/audio_chunks/audio.006/7l/7l...   
1016  ../data/raw/MusAV/audio_chunks/audio.005/1U/1U...   
658   ../data/raw/MusAV/audio_chunks/audio.002/34/34...   
1166  ../data/raw/MusAV/audio_chunks/audio.005/4u/4u...   

                           track  cosine_discogs_similarity  
1096  3RLV9wC6HBmfB3Vicwejc2.mp3                   1.000000  
1635  7l7AVAERNAeiU5SefAteOF.mp3                   0.832895  
1016  1UseeeQtmzhBVLG41JYBX0.mp3                   0.813031  
658   343SHXSYPflFvaV3A0k4xz.mp3                   0.805741  
1166  4umr7EbpUXv2KiSK3sxQlY.mp3                   0.803609  
                                               filepath  \
1096  ../data/raw/MusAV/audio_chunks/audio.005/3r/3R...   
948   ../data/raw/MusAV/audio_chunks/audio.005/5A/5A...   
1919  ../data/raw/MusAV/audio_chunks/audio.001/7e/7e...   
911   ../data/raw/MusAV/audio_chunks/